In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    roc_auc_score, f1_score, precision_score,
    recall_score, confusion_matrix, log_loss
)

data = pd.read_csv("data/initial_data.csv")

target_col = "target"

y = data[target_col].astype(int)
X = data.drop(columns=[target_col])

num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]

preproc = ColumnTransformer(
    transformers=[
        ("num", "passthrough", num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ]
)

model = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),  # with_mean=False для разреженных данных
    ("clf", LogisticRegression(solver="saga", max_iter=1000))
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model.fit(X_train, y_train)
prediction = model.predict(X_test)                # метки 0/1
probas = model.predict_proba(X_test)[:, 1]  

# prediction — это предсказанные значения модели (0/1 или вероятности)
# y_test — истинные метки

metrics = {}

# ROC-AUC
auc = roc_auc_score(y_test, probas)

# Precision, Recall, F1
precision = precision_score(y_test, prediction)
recall = recall_score(y_test, prediction)
f1 = f1_score(y_test, prediction)

# Матрица ошибок
_, err1, _, err2 = confusion_matrix(y_test, prediction, normalize='all').ravel()
# err1 = fp   # ошибка I рода
# err2 = fn   # ошибка II рода
logloss = log_loss(y_test, probas)

# Logloss (лучше подавать вероятности, а не метки!)
logloss = log_loss(y_test, prediction)

# Записываем в словарь
metrics["err1"] = err1
metrics["err2"] = err2
metrics["auc"] = auc
metrics["precision"] = precision
metrics["recall"] = recall
metrics["f1"] = f1
metrics["logloss"] = logloss

ValueError: could not convert string to float: '2014-02-01'

In [12]:
import mlflow
import os
EXPERIMENT_NAME='test_connection_experiment_vadim_shakula_1'
RUN_NAME='baseline_logreg_v1'

# создаём новый эксперимент в MLflow с указанным названием 
# если эксперимент с таким именем уже существует, 
# MLflow возвращает идентификатор существующего эксперимента
mlflow.set_experiment(EXPERIMENT_NAME)
experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    # получаем уникальный идентификатор запуска эксперимента
    run_id = run.info.run_id
    
    # логируем метрики эксперимента
    # предполагается, что переменная stats содержит словарь с метриками,
    # где ключи — это названия метрик, а значения — числовые значения метрик
    mlflow.log_metrics(stats)
    
    # логируем файлы как артефакты эксперимента — 'columns.txt' и 'users_churn.csv'
    mlflow.log_artifact('columns.txt', "dataframe")
    mlflow.log_artifact('users_churn.csv', "dataframe")


experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
# получаем данные о запуске эксперимента по его уникальному идентификатору
run = mlflow.get_run(run_id) 


# проверяем, что статус запуска эксперимента изменён на 'FINISHED'
# это утверждение (assert) можно использовать для автоматической проверки того, 
# что эксперимент был завершён успешно
assert (run.info.status =='FINISHED')

# удаляем файлы 'columns.txt' и 'users_churn.csv' из файловой системы,
# чтобы очистить рабочую среду после логирования артефактов
os.remove('columns.txt') 
os.remove('users_churn.csv')

/home/mle-user/.local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/mle-user/.local/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
2025/08/28 17:21:18 INFO mlflow.tracking.fluent: Experiment with name 'test_connection_experiment_vadim_shakula_1' does not exist. Creating a new experiment.


NameError: name 'stats' is not defined